In [ ]:
!pip3 install google-cloud-speech

     |████████████████████████████████| 92kB 3.4MB/s 


2. **Download audio files for testing**

Following files will be used as test cases for all speech recognition alternatives covered in this notebook.

In [ ]:
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.6.0/audio-0.6.0.tar.gz
!tar -xvzf audio-0.6.0.tar.gz
!ls -l ./audio/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   608    0   608    0     0   2632      0 --:--:-- --:--:-- --:--:--  2620
100  192k  100  192k    0     0   310k      0 --:--:-- --:--:-- --:--:--  310k
audio/
audio/2830-3980-0043.wav
audio/Attribution.txt
audio/4507-16021-0012.wav
audio/8455-210777-0068.wav
audio/License.txt
total 260
-rw-r--r-- 1 501 staff 63244 Nov 18  2017 2830-3980-0043.wav
-rw-r--r-- 1 501 staff 87564 Nov 18  2017 4507-16021-0012.wav
-rw-r--r-- 1 501 staff 82924 Nov 18  2017 8455-210777-0068.wav
-rw-r--r-- 1 501 staff   340 May 14  2018 Attribution.txt
-rw-r--r-- 1 501 staff 18652 May 12  2018 License.txt


3. **Define test cases**

In [ ]:
TESTCASES = [
  {
    'filename': 'audio/2830-3980-0043.wav',
    'text': 'experience proves this',
    'encoding': 'LINEAR16',
    'lang': 'en-US'
  },
  {
    'filename': 'audio/4507-16021-0012.wav',
    'text': 'why should one halt on the way',
    'encoding': 'LINEAR16',
    'lang': 'en-US'
  },
  {
    'filename': 'audio/8455-210777-0068.wav',
    'text': 'your power is sufficient i said',
    'encoding': 'LINEAR16',
    'lang': 'en-US'
  }
]

4. **Utility Functions**

In [ ]:
from typing import Tuple
import wave

def read_wav_file(filename) -> Tuple[bytes, int]:
    with wave.open(filename, 'rb') as w:
        rate = w.getframerate()
        frames = w.getnframes()
        buffer = w.readframes(frames)

    return buffer, rate

def simulate_stream(buffer: bytes, batch_size: int = 4096):
    buffer_len = len(buffer)
    offset = 0
    while offset < buffer_len:
        end_offset = offset + batch_size
        buf = buffer[offset:end_offset]
        yield buf
        offset = end_offset

## Setup

1. **Install azure speech package**

In [ ]:
!pip3 install azure-cognitiveservices-speech

     |████████████████████████████████| 3.9MB 3.4MB/s 


2. **Set service credentials**

You can enable Speech service and find credentials for your account at [Microsoft Azure portal](https://portal.azure.com/). You can open a free account [here](https://azure.microsoft.com/en-in/free/ai/).

In [ ]:
AZURE_SPEECH_KEY = 'YOUR AZURE SPEECH KEY'
AZURE_SERVICE_REGION = 'YOUR AZURE SERVICE REGION'

## Batch API

In [ ]:
import azure.cognitiveservices.speech as speechsdk

def azure_batch_stt(filename: str, lang: str, encoding: str) -> str:
    speech_config = speechsdk.SpeechConfig(
        subscription=AZURE_SPEECH_KEY,
        region=AZURE_SERVICE_REGION
    )
    audio_input = speechsdk.AudioConfig(filename=filename)
    speech_recognizer = speechsdk.SpeechRecognizer(
        speech_config=speech_config,
        audio_config=audio_input
    )
    result = speech_recognizer.recognize_once()

    return result.text if result.reason == speechsdk.ResultReason.RecognizedSpeech else None

# Run tests
for t in TESTCASES:
    print('\naudio file="{0}"    expected text="{1}"'.format(t['filename'], t['text']))
    print('azure-batch-stt: "{}"'.format(
        azure_batch_stt(t['filename'], t['lang'], t['encoding'])
    ))


audio file="audio/2830-3980-0043.wav"    expected text="experience proves this"
azure-batch-stt: "Experience proves this."

audio file="audio/4507-16021-0012.wav"    expected text="why should one halt on the way"
azure-batch-stt: "Whi should one halt on the way."

audio file="audio/8455-210777-0068.wav"    expected text="your power is sufficient i said"
azure-batch-stt: "Your power is sufficient I said."


## Streaming API

In [ ]:
import time
import azure.cognitiveservices.speech as speechsdk

def azure_streaming_stt(filename: str, lang: str, encoding: str) -> str:
    speech_config = speechsdk.SpeechConfig(
        subscription=AZURE_SPEECH_KEY,
        region=AZURE_SERVICE_REGION
    )
    stream = speechsdk.audio.PushAudioInputStream()
    audio_config = speechsdk.audio.AudioConfig(stream=stream)
    speech_recognizer = speechsdk.SpeechRecognizer(
        speech_config=speech_config,
        audio_config=audio_config
    )

    # Connect callbacks to the events fired by the speech recognizer
    speech_recognizer.recognizing.connect(lambda evt: print('interim text: "{}"'.format(evt.result.text)))
    speech_recognizer.recognized.connect(lambda evt:  print('azure-streaming-stt: "{}"'.format(evt.result.text)))

    # start continuous speech recognition
    speech_recognizer.start_continuous_recognition()

    # push buffer chunks to stream
    buffer, rate = read_wav_file(filename)
    audio_generator = simulate_stream(buffer)
    for chunk in audio_generator:
      stream.write(chunk)
      time.sleep(0.1)  # to give callback a chance against this fast loop

    # stop continuous speech recognition
    stream.close()
    time.sleep(0.5)  # give chance to VAD to kick in
    speech_recognizer.stop_continuous_recognition()
    time.sleep(0.5)  # Let all callback run

# Run tests
for t in TESTCASES:
    print('\naudio file="{0}"    expected text="{1}"'.format(t['filename'], t['text']))
    azure_streaming_stt(t['filename'], t['lang'], t['encoding'])


audio file="audio/2830-3980-0043.wav"    expected text="experience proves this"
interim text: "experience"
interim text: "experienced"
interim text: "experience"
interim text: "experience proves"
interim text: "experience proves this"
azure-streaming-stt: "Experience proves this."

audio file="audio/4507-16021-0012.wav"    expected text="why should one halt on the way"
interim text: "huaisheng"
interim text: "white"
interim text: "whi should"
interim text: "whi should one"
interim text: "whi should one halt"
interim text: "whi should one halt on"
interim text: "whi should one halt on the"
interim text: "whi should one halt on the way"
azure-streaming-stt: "Whi should one halt on the way."

audio file="audio/8455-210777-0068.wav"    expected text="your power is sufficient i said"
interim text: "you're"
interim text: "your"
interim text: "your power"
interim text: "your"
interim text: "your power is"
interim text: "your power is sufficient"
interim text: "your power is sufficient i"
int